In [40]:
%%capture

!git clone https://github.com/vinid/NegotiationArena/
!pip install -r NegotiationArena/requirements.txt

In [41]:
import sys
from dotenv import load_dotenv

sys.path.append('NegotiationArena/')

In [42]:
import os
current_path = os.getcwd()
current_path

'c:\\DATA\\Personnel\\MEGAsync\\Coursera\\ML\\ML_Learning\\negociationArena'

In [44]:
# the project assumes you have your environment variable in an env file.
load_dotenv(".env")

True

In [45]:
from negotiationarena.alternating_game import AlternatingGameEndsOnTag
from negotiationarena.parser import ExchangeGameDefaultParser
from negotiationarena.constants import (RESOURCES_TAG,
                                        MESSAGE_TAG,PLAYER_ANSWER_TAG,
                                        PROPOSED_TRADE_TAG,
                                        REASONING_TAG,
                                        ACCEPTING_TAG,
                                        AGENT_ONE,
                                        AGENT_TWO)

from negotiationarena.utils import get_tag_contents
from negotiationarena.agent_message import AgentMessage
from typing import List
from negotiationarena.agents.agents import Agent
from negotiationarena.agents import ChatGPTAgent
from negotiationarena.game_objects.resource import Resources
from glob import glob
import json

In [46]:
print(RESOURCES_TAG)
print(MESSAGE_TAG,PLAYER_ANSWER_TAG)
print(RESOURCES_TAG)
print(MESSAGE_TAG,PLAYER_ANSWER_TAG)
print(PROPOSED_TRADE_TAG)
print(REASONING_TAG)
print(ACCEPTING_TAG)
print(AGENT_ONE)
print(AGENT_TWO)


my resources
message player answer
my resources
message player answer
newly proposed trade
reason
ACCEPT
Player RED
Player BLUE


In [47]:
def simple_game_prompt(initial_resources, player_social_behaviour):
    prompt = f"""You are playing a game in which {AGENT_ONE} gives all resources to {AGENT_TWO}. {AGENT_TWO} has nothing to give.
You can only make on single trade.

RULES:


1. You must always respond with:

A) Propose a trade with (you can only trade in integer amounts, not decimals):
<player answer> PROPOSAL </player answer>
<{PROPOSED_TRADE_TAG}> {AGENT_ONE} Gives item1: amount, item2: amount ...| {AGENT_TWO} Gives item1: 0, item2: 0 </{PROPOSED_TRADE_TAG}>
Replace item names with the resources you want to exchange, for example {AGENT_ONE} Gives B: 2

B) You can only accept the trade by saying:
<player answer> {ACCEPTING_TAG} </player answer>
<{PROPOSED_TRADE_TAG}> NONE </{PROPOSED_TRADE_TAG}>

2. You can send a message to the other agent by saying:

<{MESSAGE_TAG}> your message here </{MESSAGE_TAG}>

Here is what you have access to:

<{RESOURCES_TAG}> {initial_resources} </{RESOURCES_TAG}>


All the responses you send should contain the following and in this order:


<{PLAYER_ANSWER_TAG}> [add here] </{PLAYER_ANSWER_TAG}>
<{PROPOSED_TRADE_TAG}> [add here] </{PROPOSED_TRADE_TAG}>
<{MESSAGE_TAG}> [add here] </{MESSAGE_TAG}>

Please be sure to include all.

{player_social_behaviour}
"""

    return prompt


In [48]:
print(simple_game_prompt("TO BE FILLED AT RUNTIME", "TO BE FILLED AT RUNTIME"))

You are playing a game in which Player RED gives all resources to Player BLUE. Player BLUE has nothing to give.
You can only make on single trade.

RULES:


1. You must always respond with:

A) Propose a trade with (you can only trade in integer amounts, not decimals):
<player answer> PROPOSAL </player answer>
<newly proposed trade> Player RED Gives item1: amount, item2: amount ...| Player BLUE Gives item1: 0, item2: 0 </newly proposed trade>
Replace item names with the resources you want to exchange, for example Player RED Gives B: 2

B) You can only accept the trade by saying:
<player answer> ACCEPT </player answer>
<newly proposed trade> NONE </newly proposed trade>

2. You can send a message to the other agent by saying:

<message> your message here </message>

Here is what you have access to:

<my resources> TO BE FILLED AT RUNTIME </my resources>


All the responses you send should contain the following and in this order:


<player answer> [add here] </player answer>
<newly propo

In [49]:
class SimpleGameDefaultParser(ExchangeGameDefaultParser):

    def instantiate_prompt(self, initial_resources, social_behavior):
        """
        This method is going to instantiate the prompt for each agent. It is going to
        make a player-specific prompt, telling the agent its resources and the behavior
        it should have in the game.
        """
        return simple_game_prompt(initial_resources, social_behavior)

    def parse(self, response):
        """
        This method is going to parse the raw response from an agent and return a
        structured object.

        This method is going to receive in input something like the following:

        <answer> REJECT </answer>
        <message> I don't like your trade, here's my counter offer. </message>
        <trade> Player RED Gives X:50 | Player Blue Gives Y:40 </trade>

        """
        ms = AgentMessage()

        answer = get_tag_contents(response, PLAYER_ANSWER_TAG)
        message = get_tag_contents(response, MESSAGE_TAG)

        # very important but fragile method
        trade = self.parse_trade(response, PROPOSED_TRADE_TAG)

        # all these messages are going to be sent to the other player
        ms.add_public(MESSAGE_TAG, message)
        ms.add_public(PLAYER_ANSWER_TAG, answer)
        ms.add_public(PROPOSED_TRADE_TAG, trade)

        return ms


In [50]:
class SimpleGame(AlternatingGameEndsOnTag):


    def __init__(self, players: List[Agent],
                 iterations: int,
                 player_starting_resources: List[Resources],
                 player_roles: List[str],
                 player_social_behaviour: List[str],
                 **kwargs):

        self.game_interface = SimpleGameDefaultParser()

        # some params are required by the superclass, players, number of iterations and accepting tag to end the game
        # ACCEPTING_TAG is the tag to be produced to end the game. It should appear in the <player answer> TAG
        #super().__init__(players=players, iterations=iterations, end_tag=ACCEPTING_TAG, **kwargs)
        super().__init__(players=players, iterations=iterations, **kwargs)

        #################
        # Game State    #
        #################

        self.player_initial_resources = player_starting_resources
        self.player_roles = player_roles
        self.player_social_behaviour = player_social_behaviour

        ####################################
        # Adding Some Logging Information  #
        ####################################

        self.game_state: List[dict] = [
            {
                "current_iteration": "START",
                "turn": "None",
                "settings": dict(
                    player_initial_resources=self.player_initial_resources,
                    player_roles=self.player_roles,
                    player_social_behaviour=self.player_social_behaviour,
                ),
            }
        ]

        # init players
        self.init_players()

    def init_players(self):
        settings = self.game_state[0]["settings"]

        #################
        # Agent Setup   #
        #################

        # Now we have to tell each GPT agent of its role
        # for each player
        for idx, player in enumerate(self.players):

            # we instantiate a player specific prompt, meaning that
            # each agent is going to have it's own prompt with its own resources

            game_prompt = self.game_interface.instantiate_prompt(
                initial_resources=settings["player_initial_resources"][idx],
                social_behavior=settings["player_social_behaviour"][idx],
            )

            player.init_agent(game_prompt, role=settings["player_roles"][idx])


    def after_game_ends(self):
        datum = dict(current_iteration="END", turn="None", summary=dict())

        self.game_state.append(datum)


In [51]:
a1 = ChatGPTAgent(
    model="gpt-4-1106-preview",
    agent_name=AGENT_ONE,
)
a2 = ChatGPTAgent(
    model="gpt-4-1106-preview",
    agent_name=AGENT_TWO,
)

c = SimpleGame(
    players=[a1, a2],
    iterations=6,
    player_starting_resources=[
        Resources({"X": 25, "Y": 5}),
        Resources({"X": 0, "Y": 0}),
    ],
    player_social_behaviour=[
        "",
        "",
    ],
    player_roles=[
        f"You are {AGENT_ONE}, start by making a proposal.",
        f"You are {AGENT_TWO}, start by accepting a trade.",
    ],
    log_dir="./.logs/simple_game/",
)

c.run()


AttributeError: 'NoneType' object has no attribute 'instantiate_prompt'